In [1]:
%pip install --quiet bitsandbytes==0.45.0 transformers==4.47.1 accelerate==1.2.1 sentencepiece==0.2.0 optimum==1.23.3 auto-gptq==0.7.1 torchmetrics

import gc
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import bitsandbytes as bnb
import subprocess
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from IPython.display import clear_output
from random import sample
from tqdm import tqdm
from torchmetrics.functional import accuracy
from torch.optim import AdamW
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig

sns.set_theme()

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import re
import ast
from typing import List, Dict, Any

def parse_all_data(file_path: str) -> List[Dict[str, Any]]:
    """Парсинг всех данных без учета времени ответа"""
    return _parse_data(file_path, include_time=False)

def parse_data_with_time(file_path: str) -> List[Dict[str, Any]]:
    """Парсинг данных с сохранением времени ответа"""
    return _parse_data(file_path, include_time=True)

def _parse_data(file_path: str, include_time: bool) -> List[Dict[str, Any]]:
    """Базовая функция парсинга"""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    result = []
    for item in data:
        parsed = {
            'selected_role': item['Выбранная роль'],
            'campus': item['Кампус'],
            'education_level': item['Уровень образования'],
            'question_category': item['Категория вопроса'],
            'user_question': _clean_text(item['Вопрос пользователя']),
            'user_filters': item['user_filters'],
            'question_filters': item['question_filters'],
            'saiga_answer': _clean_text(item['Saiga']),
            'giga_answer': _clean_text(item['Giga']),
            'winner': item['Кто лучше?'],
            'comment': item['Комментарий'],
            'contexts': _parse_contexts(item['Ресурсы для ответа'])
        }

        if item.get('Уточненный вопрос пользователя'):
            parsed.update({
                'refined_question': _clean_text(item['Уточненный вопрос пользователя']),
                'refined_answer': _clean_text(item['Ответ AI (уточнение)']),
                'refined_contexts': _parse_contexts(item['Ресурсы для ответа (уточнение)'] or '')
            })

        if include_time:
            parsed.update({
                'response_time': item['Время ответа модели (сек)'],
                'refined_response_time': item.get('Время ответа модели на уточненный вопрос (сек)')
            })

        result.append(parsed)

    return result

def _parse_contexts(resources: str) -> List[Dict[str, Any]]:
    """Парсинг контекстов с использованием вашей функции"""
    contexts = []
    pattern = re.compile(r"Document\(page_content='(.*?)', metadata=({.*?})\)", re.DOTALL)

    for match in re.finditer(pattern, resources):
        content, metadata_str = match.groups()
        try:
            metadata = ast.literal_eval(metadata_str)
            tags = _extract_tags(metadata)

            contexts.append({
                'text': _clean_text(content),
                'metadata': {
                    'source': metadata.get('source'),
                    'file_name': metadata.get('file_name'),
                    'url': metadata.get('url')
                },
                'tags': tags
            })
        except Exception as e:
            print(f"Контекст не распарсился: {e}")

    return contexts

def _extract_tags(metadata: Dict) -> Dict[str, List[str]]:
    """Извлечение тегов в отдельные категории"""
    return {
        'topic_tags': [v for k,v in metadata.items() if k.startswith('topic_tag_') and v],
        'user_tags': [v for k,v in metadata.items() if k.startswith('user_tag_') and v]
    }

def _clean_text(text: str) -> str:
    """Очистка текста"""
    if not text: return ''
    return re.sub(r'\\[nrt]|[\n\r\t]+|\s+', ' ', text).strip()

In [8]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=db04229f541d4a575505923d0f809ea701c23b5252e8a1631eefc69e28f6d44f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [14]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.4 MB/s eta 0:00:00


In [16]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00


In [17]:
from typing import List

import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")
bertscore = evaluate.load("bertscore")


def context_recall(ground_truth: str, contexts: List[str])->float:
    """
    Calc rouge btw contexts and ground truth.
    Interpretation: ngram match (recall) btw contexts and desired answer.

    ROUGE - https://huggingface.co/spaces/evaluate-metric/rouge

    return: average rouge for all contexts.
    """
    rs = []
    for c in contexts:
        rs.append(
            rouge.compute(
                predictions=[str(c)],
                references=[str(ground_truth)],
            )["rouge2"]
        )

    return np.mean(rs)


def context_precision(ground_truth: str, contexts: List[str])->float:
    """
    Calc blue btw contexts and ground truth.
    Interpretation: ngram match (precision) btw contexts and desired answer.

    BLEU - https://aclanthology.org/P02-1040.pdf
    max_order - max n-grams to count

    return: average bleu (precision2, w/o brevity penalty) for all contexts.
    """
    bs = []
    for c in contexts:

        try:
            bs.append(
                bleu.compute(
                    predictions=[str(c)],
                    references=[str(ground_truth)],
                    max_order=2,
                )["precisions"][1]
            )
        except ZeroDivisionError:
            bs.append(0)

    return np.mean(bs)


def answer_correctness_literal(
    ground_truth: str,
    answer: str,
    char_order: int = 6,
    word_order: int = 2,
    beta: float = 1,
)->float:
    """
    Calc chrF btw answer and ground truth.
    Interpretation: lingustic match btw answer and desired answer.

    chrF - https://aclanthology.org/W15-3049.pdf
    char_order - n-gram length for chars, default is 6 (from the article)
    word_order - n-gram length for words (chrF++), default is 2 (as it outperforms simple chrF)
    beta - recall weight, beta=1 - simple F1-score

    return: chrF for answ and gt.
    """

    score = chrf.compute(
        predictions=[str(answer)],
        references=[str(ground_truth)],
        word_order=word_order,
        char_order=char_order,
        beta=beta,
    )["score"]

    return score


def answer_correctness_neural(
    ground_truth: str,
    answer: str,
    model_type: str = "cointegrated/rut5-base",
)->float:
    """
    Calc bertscore btw answer and ground truth.
    Interpretation: semantic cimilarity btw answer and desired answer.

    BertScore - https://arxiv.org/pdf/1904.09675.pdf
    model_type - embeds model  (default t5 as the best from my own research and experience)

    return: bertscore-f1 for answ and gt.
    """

    score = bertscore.compute(
        predictions=[str(answer)],
        references=[str(ground_truth)],
        batch_size=1,
        model_type=model_type,
        num_layers=11,
    )["f1"]

    return score


class ValidatorSimple:
    """
    Расчет простых метрик качества для заданного датасета.
    """
    def __init__(
        self,
        neural: bool = False,
    ):
        """
        param neural: есть гпу или нет. По дефолту ее нет(
        """
        self.neural = neural

    def score_sample(
        self,
        answer: str,
        ground_truth: str,
        context: List[str],
    ):
        """
        Расчет для конкретного сэмпла в тестовом датасете.
        """
        scores = {}
        scores["context_recall"] = [
            context_recall(
                ground_truth,
                context,
            )
        ]
        scores["context_precision"] = [
            context_precision(
                ground_truth,
                context,
            )
        ]
        scores["answer_correctness_literal"] = [
            answer_correctness_literal(
                ground_truth=ground_truth,
                answer=answer,
            )
        ]
        if self.neural:
            scores["answer_correctness_neural"] = [
                answer_correctness_neural(
                    ground_truth=ground_truth,
                    answer=answer,
                )
            ]
        return scores

    def validate_rag(
        self,
        test_set: pd.DataFrame,
    ):
        """
        param test_set: пандас датасет с нужными полями: answer, ground_truth, context, question
        """

        res = {}
        for _, row in tqdm(test_set.iterrows(), "score_sample"):
            gt = row.ground_truth
            answer = row.answer
            context = row.contexts
            scores = self.score_sample(answer, gt, context)
            if not res:
                res = scores
            else:
                for k, v in scores.items():
                    res[k].extend(v)
        for k, v in res.items():
            res[k] = np.mean(res[k])
        return res

In [19]:

data_v1 = parse_all_data('val_set.json')

data_v2 = parse_data_with_time('val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

In [20]:
import json

with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)